In [151]:
import os
import json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.ndimage as snd

In [152]:
def rgb_to_intensity(rgb_image):
    return np.dot(rgb_image[...,:3], [0.299, 0.587, 0.114])

In [167]:
def get_train_images(directories, need_print: bool = False, get_full: bool = False):
    for directory in directories:
        if need_print:
            print(directory)

        found_images = [file_name for file_name in os.listdir(os.path.join(directory, 'img')) if file_name.endswith('.png') or file_name.endswith('.jpg')]
        for file_name in found_images:
            if need_print:
                print(file_name, end=' ')

            if get_full:
                yield (
                    os.path.join(directory, 'img', file_name),
                    os.path.join(directory, 'ann', file_name+'.json'),
                    os.path.join(directory, 'masks_machine', file_name),
                    directory.split('/')[-1],
                )
            else:
                yield os.path.join(directory, 'img', file_name)

Считаем что-то более простое (и быстрое)

In [154]:
def count_rods(json_path: str):
    with open(json_path) as mask_json:
        mask_content = mask_json.read()
        json_dict = json.loads(mask_content)
    
    return len(json_dict['objects'])

def get_total_rods_area_ratio(mask_path: str):
    mask_image = mpimg.imread(mask_path)
    meaned_mask = np.mean(mask_image, axis=2)
    binarized_mask = np.where(meaned_mask != 0, 1, 0)
    return np.mean(binarized_mask)

def get_intensity_mean(img_path: str):
    rgb_image = mpimg.imread(img_path)
    image_intensity = rgb_to_intensity(rgb_image)
    return np.mean(image_intensity)

def get_intensity_var(img_path: str):
    rgb_image = mpimg.imread(img_path)
    image_intensity = rgb_to_intensity(rgb_image)
    return np.mean(np.power(image_intensity - np.mean(image_intensity), 2))

In [155]:
print(get_total_rods_area_ratio('data_old_separated/train/masks_machine/IMG_1748.png'))
print(count_rods('data_old_separated/train/ann/IMG_1748.png.json'))
print(get_intensity_mean('data_old_separated/train/img/IMG_1748.png'))
print(get_intensity_var('data_old_separated/train/img/IMG_1748.png'))

0.11892320103195599
8
0.4873936248487635
0.04721309090745074


Оптимизируем STA6 с помощью сверток прямо из библиотеки (метрику считаем только по пикселям маски)

In [156]:
def sta6_optimized(img_path: str, conv_size: int = 0, stride: int = 1):
    rgb_image = mpimg.imread(img_path)
    image_intensity = rgb_to_intensity(rgb_image)

    mean_kernel = np.ones((conv_size, conv_size)) / conv_size**2
    image_mean_intensity = snd.convolve(image_intensity, mean_kernel)[:image_intensity.shape[0], :image_intensity.shape[1]]

    image_intensity = image_intensity[::stride, ::stride]
    image_mean_intensity = image_mean_intensity[::stride, ::stride]
    
    sta6 = np.sum(np.power(image_intensity - image_mean_intensity, 2)) / image_intensity.size

    return sta6

In [168]:
# dirs = ['data/train', 'data/bad']
dirs = ['data/appropriate', 'data/bad']

names = []
folders = []
counts = []
ratios = []
int_means = []
int_vars = []

for image_name, ann_name, mask_name, folder_name in get_train_images(dirs, get_full=True):
    names.append(image_name.split('/')[-1])
    folders.append(folder_name)
    counts.append(count_rods(ann_name))
    ratios.append(get_total_rods_area_ratio(mask_name))
    int_means.append(get_intensity_mean(image_name))
    int_vars.append(get_intensity_var(image_name))

df = pd.DataFrame.from_dict({
    'Index': names,
    'Dataset': folders,
    'Rod Count': counts,
    'Area Ratio': ratios,
    'Intensity Mean': int_means,
    'Intensity Variance': int_vars,
})

In [171]:
df[df['Dataset'] == 'appropriate']

,Index,Dataset,Rod Count,Area Ratio,Intensity Mean,Intensity Variance
0,IMG_6410.png,appropriate,11,0.368527,0.508420,0.071992
1,IMG_6365.png,appropriate,7,0.172307,0.525506,0.055674
2,IMG_6497.png,appropriate,2,0.789966,0.545624,0.023082
3,IMG_1753.png,appropriate,8,0.158387,0.463716,0.058269
4,IMG_1760.png,appropriate,37,0.194448,0.433240,0.061916
5,IMG_6362.png,appropriate,7,0.461145,0.509379,0.043082
6,IMG_1752.png,appropriate,8,0.167471,0.482879,0.046782
7,IMG_6495.png,appropriate,3,0.755973,0.475130,0.025412
8,IMG_1758.png,appropriate,32,0.243413,0.451460,0.060151
9,IMG_6406.png,appropriate,6,0.564901,0.486565,0.036616


In [172]:
df[df['Dataset'] == 'bad']

,Index,Dataset,Rod Count,Area Ratio,Intensity Mean,Intensity Variance
30,IMG_6369.png,bad,10,0.430101,0.446417,0.045606
31,IMG_6417.png,bad,10,0.443148,0.503094,0.032830
32,IMG_6375.png,bad,5,0.540432,0.497355,0.043422
33,IMG_6376.png,bad,12,0.238160,0.517558,0.035138
34,IMG_6415.png,bad,21,0.278510,0.457350,0.037116
35,IMG_6364.png,bad,20,0.230936,0.381220,0.045219
36,IMG_6408.png,bad,78,0.361267,0.502969,0.051043
37,IMG_6416.png,bad,24,0.310809,0.502227,0.031971
38,IMG_6374.png,bad,16,0.407425,0.474951,0.045328
39,IMG_6422.png,bad,28,0.171472,0.503551,0.040003
